In [51]:
# This is the project code.

In [52]:
import pyspark
from delta import *

builder = pyspark.sql.SparkSession.builder.appName("DF2_Practice") \
    .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension") \
    .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog") 

spark = configure_spark_with_delta_pip(builder).getOrCreate()

#spark.conf.set("spark.sql.repl.eagerEval.enabled",True) # OK for exploration, not great for performance
#spark.conf.set("spark.sql.repl.eagerEval.truncate", 500)

In [53]:
import pandas as pd
import json
import numpy as np
import shapely
from shapely import Point
from shapely.geometry import mapping, shape
import pyspark.sql.functions as F
from pyspark.sql import Window
from shapely import Polygon
from pyspark.sql.functions import col, count, when
from pyspark.sql.types import DoubleType, IntegerType, StringType
from shapely.strtree import STRtree

## Reading in data

In [54]:
# Reading in taxi dataset
taxiDataDf = (spark.read
             .option("sep", ",") # separator
             .option("header", True) # file has header row
             .option("inferSchema", True) # spark tries to infer data types
             .csv("trip_data/trip_data_1.csv") #path
            ) \
            .dropna(subset=["pickup_datetime","dropoff_datetime","pickup_longitude","pickup_latitude","dropoff_longitude","dropoff_latitude"])

In [55]:
display(taxiDataDf)

DataFrame[medallion: string, hack_license: string, vendor_id: string, rate_code: int, store_and_fwd_flag: string, pickup_datetime: timestamp, dropoff_datetime: timestamp, passenger_count: int, trip_time_in_secs: int, trip_distance: double, pickup_longitude: double, pickup_latitude: double, dropoff_longitude: double, dropoff_latitude: double]

In [56]:
# Reading in borough dataset
with open('nyc-boroughs.geojson') as file:
    boroughs = json.load(file)

features = boroughs['features']
properties = [feature['properties'] for feature in features]
geometry = [feature['geometry'] for feature in features]

rowNumberWindow = Window.orderBy(F.lit("a"))

properties_df = spark.createDataFrame(properties).withColumn("rid",F.row_number().over(rowNumberWindow))
geometry_df = spark.createDataFrame(geometry).withColumn("rid",F.row_number().over(rowNumberWindow))
borough_base_df = properties_df.join(geometry_df,("rid")).drop("rid") #rid - row_id

In [57]:
display(borough_base_df)

DataFrame[@id: string, borough: string, boroughCode: bigint, coordinates: array<array<array<double>>>, type: string]

In [58]:
# Finding the area size of the boroughs
def calculate_area(coordinates):
    polygon = Polygon(list(coordinates[0]))
    return polygon.area

calculate_area_udf = F.udf(calculate_area,DoubleType())

borough_df_sorted = borough_base_df.withColumn("area",calculate_area_udf(F.col("coordinates"))) \
                            .sort(F.col("area").desc()) \
                            .withColumn("row_id",F.row_number().over(rowNumberWindow) - 1)


# https://shapely.readthedocs.io/en/stable/strtree.html
geomtree = STRtree([Polygon(coords["coordinates"][0]) for coords in borough_df_sorted.select("coordinates").collect()])
borough_df_sorted

DataFrame[@id: string, borough: string, boroughCode: bigint, coordinates: array<array<array<double>>>, type: string, area: double, row_id: int]

In [59]:
def findBorough(x,y):
    point = Point(x,y)
    nearestLocIndex = geomtree.nearest(point)
    return nearestLocIndex

In [60]:

# Define a function to find the point borough
def update_borough(longitude, latitude):
    borough = findBorough(longitude, latitude)
    return int(borough) if borough is not None else -1

# Create a user-defined function (UDF)
update_borough_udf = F.udf(update_borough, IntegerType())

# Apply the UDF to update the start and end borough column (finding the borough index)
taxiDataDf_with_boroughs = taxiDataDf.withColumn("startBoroughIndex", update_borough_udf(taxiDataDf["pickup_longitude"], taxiDataDf["pickup_latitude"]))
taxiDataDf_with_boroughs = taxiDataDf_with_boroughs.withColumn("endBoroughIndex", update_borough_udf(taxiDataDf["dropoff_longitude"], taxiDataDf["dropoff_latitude"]))


In [61]:
taxiDataDf_with_boroughs.groupBy("startBoroughIndex").count().orderBy("startBoroughIndex")

DataFrame[startBoroughIndex: int, count: bigint]

## Query 1: Utilization

Utilization: This is per taxi/driver. This can be computed by computing the idle 
time per taxi. We will elaborate on that more later

In [62]:
taxiUtilizationBaseDf = taxiDataDf_with_boroughs.filter("trip_time_in_secs > 0 or trip_time_in_secs <= 3600 * 4")
              

utilizationWindow = Window.partitionBy("hack_license").orderBy("pickup_datetime")

idleTimeDf = taxiUtilizationBaseDf.withColumn("idle_time",(F.col("pickup_datetime") - F.lag(F.col("dropoff_datetime")).over(utilizationWindow)).cast("long")) \
                                  .fillna(0,subset=["idle_time"])
display(idleTimeDf)

DataFrame[medallion: string, hack_license: string, vendor_id: string, rate_code: int, store_and_fwd_flag: string, pickup_datetime: timestamp, dropoff_datetime: timestamp, passenger_count: int, trip_time_in_secs: int, trip_distance: double, pickup_longitude: double, pickup_latitude: double, dropoff_longitude: double, dropoff_latitude: double, startBoroughIndex: int, endBoroughIndex: int, idle_time: bigint]

In [63]:
query1_df = idleTimeDf.groupBy("hack_license").agg(F.sum("idle_time").alias("idle_time_sum_seconds"))
display(query1_df)

DataFrame[hack_license: string, idle_time_sum_seconds: bigint]

In [64]:
#Save result to file
query1_df.coalesce(1).write.mode("overwrite").csv("results/m1/query1.csv",header=True)

## Query 2: Average next trip time

The average time it takes for a taxi to find its next fare(trip) per destination borough. This can be computed by finding the time difference, e.g. in seconds, between the trip's drop off and the next trip's pick up.


In [65]:
from pyspark.sql.functions import col, unix_timestamp, lead, avg, round

windowSpec = Window.partitionBy("hack_license").orderBy("dropoff_datetime")

df = taxiDataDf_with_boroughs.withColumn("next_pickup_time", lead("pickup_datetime").over(windowSpec))
df = df.withColumn("time_to_next_fare", unix_timestamp("next_pickup_time") - unix_timestamp("dropoff_datetime"))
df = df.filter((col("time_to_next_fare") >= 0) & (col("time_to_next_fare") <= 3600*4))  

result_df = df.join(borough_df_sorted, df.endBoroughIndex == borough_df_sorted.row_id)
avg_wait_time_per_borough = result_df.groupBy("borough").agg(round(avg("time_to_next_fare") / 60 , 2).alias("avg_waiting_time"))

avg_wait_time_per_borough.show()

+-------------+----------------+
|      borough|avg_waiting_time|
+-------------+----------------+
|       Queens|           31.58|
|     Brooklyn|           22.61|
|Staten Island|           40.39|
|    Manhattan|           10.39|
|        Bronx|           33.57|
+-------------+----------------+



## Query 3: Trips started in one borough

The number of trips that started and ended within the same borough,


In [66]:
same_borough_df = taxiDataDf_with_boroughs.filter(F.col("startBoroughIndex") == F.col("endBoroughIndex"))
print("The number of trips that start and end in the same borough is " + str(same_borough_df.count()) + ".")

The number of trips that start and end in the same borough is 13118340.


## Query 4: Trips between different boroughs

The number of trips that started in one borough and ended in another one

In [67]:
different_borough_df = taxiDataDf_with_boroughs.filter(F.col("startBoroughIndex") != F.col("endBoroughIndex"))
print("The number of trips that have different start and end borough is " + str(different_borough_df.count()) + ".")

The number of trips that have different start and end borough is 1658189.
